In [1]:
from residual_units import residual_unit
import tensorflow as tf
import keras
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.layers import Input
from keras.utils.vis_utils import model_to_dot
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from pickle import load
import matplotlib.pyplot as plt
from keras import optimizers
from pickle import dump
import numpy as np
import os

from models import ARL92, NAL92, ARL56, NAL56, channel_att56, inception_attention

Using TensorFlow backend.
/Users/jianfenglin/anaconda3/envs/envTF113/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jianfenglin/anaconda3/envs/envTF113/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jianfenglin/anaconda3/envs/envTF113/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1

In [2]:
# load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [3]:
## To record training loss and accuracy at each batch

##reference: https://keras.io/callbacks/

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.accs = []
#         self.val_losses = []
#         self.val_accs = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.accs.append(logs.get('accuracy'))
#         self.val_accs.append(logs.get('val_acc'))
#         self.val_losses.append(logs.get("val_loss"))
        
history_r = LossHistory()

In [4]:
## choose the name of model to switch the model
model_name = 'ARL92'
# model_name = 'NAL92'
# model_name = "ARL56"
# model_name = "NAL56"
# model_name = "channel_att56"
# model_name = "inception_attention"

## save model at each epoch
if not os.path.exists('Result/{}'.format(model_name)):
    os.makedirs('Result/{}/Weight'.format(model_name))
    os.makedirs('Result/{}/History'.format(model_name))
checkpoint = ModelCheckpoint('Result/{}/Weight/best_weights.h5'.format(model_name), save_best_only=True, save_weights_only=True, period=1)


In [5]:
## data augment
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True)

datagen.fit(x_train)
val_datagen.fit(x_train)

if model_name == 'ARL92':
    model = ARL92(input = x_train,n_classes=10)
elif model_name == 'NAL92':
    model = NAL92(input = x_train,n_classes=10)
elif model_name == 'ARL56':
    model = ARL56(input = x_train,n_classes=10)
elif model_name == 'NAL56':
    model = NAL56(input = x_train,n_classes=10)
elif model_name == 'channel_att56':
    model = channel_att56(input = x_train,n_classes=10)
elif model_name == 'inception_attention':
    model = inception_attention(input = x_train,n_classes=10)

## the author did not provide any information about the callback e.g. dropout or early stop. so we did not apply 
## any of these tricks to keep the result clean
model.compile(keras.optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

batch_size = 32

history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train) / batch_size,
                    validation_data=val_datagen.flow(x_test, y_test, batch_size=batch_size), 
                    validation_steps=len(x_test)//batch_size,
                    epochs=100,callbacks=[history_r, checkpoint])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100


KeyboardInterrupt: 

In [ ]:
## save the model histry
with open('Result/{}/History/train_lossHistory'.format(model_name), 'wb') as handle: # saving the history of the model training loss
    dump(model.history.history.get("loss"), handle)
with open('Result/{}/History/train_accHistory'.format(model_name), 'wb') as handle: # saving the history of the model training accuracy
    dump(model.history.history.get("accuracy"), handle)
with open('Result/{}/History/vali_lossHistory'.format(model_name), 'wb') as handle: # saving the history of the model training loss
    dump(model.history.history.get("val_loss"), handle)
with open('Result/{}/History/vali_accHistory'.format(model_name), 'wb') as handle: # saving the history of the model training accuracy
    dump(model.history.history.get("val_accuracy"), handle)
    

In [ ]:
# Plot training & validation accuracy values
plt.plot(model.history.history.get("loss"))
plt.plot(model.history.history.get("val_loss"))
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(model.history.history.get("accuracy"))
plt.plot(model.history.history.get("val_accuracy"))
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation accuracy values
plt.plot(history_r.losses)
plt.title('Model accuracy')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.show()

# Plot training & validation accuracy values
plt.plot(history_r.accs)
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.show()

In [ ]:
result = model.evaluate_generator(val_datagen.flow(x_test, y_test))

In [ ]:
result